In [6]:
import numpy as np
from matplotlib import pyplot as plt
from normal_normal_model import get_samples
from covariance_CI_test import CovYrZsCITest, CovXIrXJsCITest, CovV1V2CITest

In [7]:

N = 500
M = 100
mu_X = 0
mu_epsilon = 0
sigma_sq_X = 1.5**2
sigma_sq_1 = 1.1**2
sigma_sq_2 = 0.6**2

r = 450
s = 485

In [32]:
num_runs = 379
num_covariate_samples = 163
num_tests = 100

In [33]:
def print_covariance_CI_test_results(covariance_CI_tests, test_name:str):
    within_CI = [test.theoretical_quantity_in_sample_CI()
                 for test in covariance_CI_tests]
    
    print(test_name + 
          ": {}/{} ({}%) "
          .format(np.sum(within_CI), len(within_CI), 
                  np.round(100.0*np.sum(within_CI)/len(within_CI), 2)) +
          "of the tests have the theoretical quantity within the CI.")

In [ ]:
cov_Yr_Zs_tests = []
cov_XIr_XJs_tests = []
cov_V1_V2_tests = []

for num_test in range(0, num_tests):
    try:
        # Sample the parameters from a realistic parameter space
        N = int(10 ** np.random.uniform(1.0, 3.6))
        M = int(N* np.random.uniform(0.01, 0.99))
        sigma_sq_X = np.random.uniform(0.3, 10) ** 2
        sigma_sq_1 = np.random.uniform(0.3, 10) ** 2
        sigma_sq_2 = max((np.sqrt(sigma_sq_1) * np.random.uniform(0.1, 0.99)) ** 2,
                         0.2 ** 2)
        r = int(np.random.uniform(N-M+1, N))
        s = int(np.random.uniform(N-M+1, N))
        
        # Prepare a test
        cov_Yr_Zs_test = CovYrZsCITest(r, s, sigma_sq_X, sigma_sq_1, sigma_sq_2, N)
        cov_XIr_XJs_test = CovXIrXJsCITest(r, s, sigma_sq_X, sigma_sq_1, sigma_sq_2, N)
        cov_V1_V2_test = CovV1V2CITest(sigma_sq_X, sigma_sq_1, sigma_sq_2, N, M)

        for num_covariate_sample in range(0, num_covariate_samples):
            print("Test {}/{} (N={}, M={}): calculating sample {}/{}...      "
                  .format(num_test+1, num_tests, N, M,
                          num_covariate_sample+1, num_covariate_samples), 
                  end='\r')

            # Generate samples from multiple runs to obtain one
            # covariate sample for each pair of r.v. we are intersted in
            samples = get_samples(
                num_runs, N, M, mu_X, mu_epsilon, 
                sigma_sq_X, sigma_sq_1, sigma_sq_2,
                verbose=False, r=r, s=s)

            for test in [cov_Yr_Zs_test, cov_XIr_XJs_test, cov_V1_V2_test]:
                test.add_sample(samples)
        
        # Once a test is completed (i.e. we collected enough covariate
        # samples), we added them to the existing collection of tests
        cov_Yr_Zs_tests.append(cov_Yr_Zs_test)
        cov_XIr_XJs_tests.append(cov_XIr_XJs_test)
        cov_V1_V2_tests.append(cov_V1_V2_test)
        
    except KeyboardInterrupt:
        # Breaking the for loop is sufficient, as we want to
        # analyse the results accumulated so far
        break
        
print_covariance_CI_test_results(cov_Yr_Zs_tests, 'Cov(Y_(r), Z_(s))')
print_covariance_CI_test_results(cov_XIr_XJs_tests, 'Cov(Y_I(r), Z_J(s))')
print_covariance_CI_test_results(cov_V1_V2_tests, 'Cov(V_1, V_2)')

# TODO: Save the tests as a pickle for future reference.
# TODO: Convert this to a callable program https://stackabuse.com/command-line-arguments-in-python/